# 3.1 Teilen Sie Ihren Datensatz in einen test- und einen train-Satz ein. 

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error, r2_score

file_path = "daten metacritic.csv"


df = pd.read_csv(file_path, sep=';')
df['r-date'] = pd.to_datetime(df['r-date'], format='%d.%m.%Y')
df['r-date'] = df['r-date'].astype('int64') // 10**9
 
print(f"Loaded {len(df)} rows of data")
print(df.head(2))

features = ['score', 'r-date', 'critics', 'users', '1 Player'	,'1-2 Player',	'No Online Multiplayer'	,'MP up to 32'	,'MP up to 30',	'MP up to 16'	,'No info'	,'1-4 Player'] 

X = df[features]
y = df['user score'] 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(df[features].dtypes)
print(f"Training data: {X_train.shape[0]} samples")
print(f"Test data: {X_test.shape[0]} samples")

Loaded 16658 rows of data
            name      platform      r-date  score  user score developer  \
0  Madden NFL 21  PlayStation4  1598313600     63         0.2  EASports   
1  Madden NFL 21       XboxOne  1598313600     69         0.4  EASports   

                      genre   players  critics  users  1 Player  1-2 Player  \
0  Sports,Team,Football,Sim  Up to 6        29   5363         0           0   
1  Sports,Team,Football,Sim  Up to 6        20    994         0           0   

   No Online Multiplayer  MP up to 32  MP up to 30  MP up to 16  No info  \
0                      0            0            0            0        0   
1                      0            0            0            0        0   

   1-4 Player  
0           0  
1           0  
score                    int64
r-date                   int64
critics                  int64
users                    int64
1 Player                 int64
1-2 Player               int64
No Online Multiplayer    int64
MP up to 32     

# 3.2 Bestimmen Sie einen Algorithmus aus sklearn, der für Ihren Datensatz und Ihr Problem geeignet ist. Erklären Sie Ihre Entscheidung in 50 bis 100 Wörtern. Alternativ dazu können Sie auch verschiedene Algorithmen evaluieren und so bestimmen, welcher am besten geeignet ist. Berechnen Sie ein Modell mittels Ihres ausgesuchten Algorithmus auf Ihren train-Daten.

In [7]:

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

results = {}
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    train_rmse = root_mean_squared_error(y_train, train_pred)
    test_rmse = root_mean_squared_error(y_test, test_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    results[name] = {"Train RMSE": train_rmse, "Test RMSE": test_rmse, "Test R²": test_r2}
    print(f"{name}: Train RMSE = {train_rmse:.4f}, Test RMSE = {test_rmse:.4f}, Test R² = {test_r2:.4f}")

best_model = min(results, key=lambda x: results[x]["Test RMSE"])
print(f"\nBester Algorithmus: {best_model}")

Training Linear Regression...
Linear Regression: Train RMSE = 1.0740, Test RMSE = 1.0995, Test R² = 0.3597
Training Ridge Regression...
Ridge Regression: Train RMSE = 1.0740, Test RMSE = 1.0994, Test R² = 0.3599
Training Random Forest...


C:\Users\kamil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=8.75244e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Random Forest: Train RMSE = 0.3960, Test RMSE = 1.0443, Test R² = 0.4224
Training Gradient Boosting...
Gradient Boosting: Train RMSE = 1.0115, Test RMSE = 1.0306, Test R² = 0.4375

Bester Algorithmus: Gradient Boosting


# 3.3 Testen Sie, wie gut Ihr Modell funktioniert, indem Sie einige Vorhersagen aus dem test-Datensatz generieren und diese manuell auf ihre Sinnhaftigkeit überprüfen. Fassen Sie Ihre Erkenntnisse in 50 bis 100 Wörtern zusammen.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import joblib

np.random.seed(42)

file_path = "daten metacritic.csv"

df = pd.read_csv(file_path, sep=';')
df['r-date'] = pd.to_datetime(df['r-date'], format='%d.%m.%Y')
df['r-date'] = df['r-date'].astype('int64') // 10**9

features = ['score', 'r-date', 'critics', 'users', '1 Player', '1-2 Player', 'No Online Multiplayer', 'MP up to 32', 'MP up to 30', 'MP up to 16', 'No info', '1-4 Player'] 

X = df[features]
y = df['user score'] 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)

joblib.dump(gb_model, 'metacritic_model.joblib')

sample_size = 200
np.random.seed(None)
random_indices = np.random.choice(len(X_test), sample_size, replace=False)
np.save('sample_indices.npy', random_indices)

sample_X = X_test.iloc[random_indices]
sample_y = y_test.iloc[random_indices]
sample_names = df.iloc[sample_X.index]['name']
sample_platforms = df.iloc[sample_X.index]['platform']
sample_scores = df.iloc[sample_X.index]['score']

predictions = gb_model.predict(sample_X)

comparison = pd.DataFrame({
    'Game': sample_names,
    'Platform': sample_platforms, 
    'Critic Score': sample_scores,
    'Actual User Score': sample_y,
    'Predicted User Score': predictions,
    'Error': np.abs(sample_y - predictions)
})

print("\nSample Predictions vs Actual Values:")
print(comparison.round(2))

avg_error = comparison['Error'].mean()
print(f"\nAverage Absolute Error: {avg_error:.2f}")


Sample Predictions vs Actual Values:
                                          Game      Platform  Critic Score  \
9246   Naruto Shippuden: Ultimate Ninja Impact           PSP            59   
8748                                   F1 2017       XboxOne            84   
409                       Game Party Champions          WiiU            24   
251                             Torchlight III            PC            65   
14857                           Dragon's Dogma  PlayStation3            78   
...                                        ...           ...           ...   
9060                               God Eater 3  PlayStation4            71   
6507                               ShadowFlare            PC            49   
5270                             Kakuto Chojin          Xbox            46   
15614                        Midtown Madness 2            PC            78   
9851                               NBA Live 07           PSP            64   

       Actual User Score 

Nach Überprüfung der Vorhersagen des Gradient Boosting Modells zeigt sich, dass es die Benutzerscores mit ordentlicher Genauigkeit vorhersagt. Bei Spielen mit extremen Bewertungsunterschieden zwischen Kritikern und Nutzern sind die Fehler grösser. Das Modell tendiert dazu, bei hochgelobten Spielen zu optimistisch und bei stark kritisierten Spielen zu pessimistisch zu sein. Die Vorhersagegenauigkeit ist am besten bei Spielen mit mittleren Bewertungen. Insbesondere kann das Modell gut erkennen, wenn ein Spiel von Nutzern deutlich schlechter bewertet wird als von Kritikern. Dies deutet darauf hin, dass die Faktoren, die zur Nutzerunzufriedenheit führen, konsistente Muster aufweisen.